In [25]:
import pandas as pd

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [2]:
path = 'D:/Disco/Data/PUCRS/projeto CDIA II/'

In [3]:
cons_reg = pd.read_csv(f'{path}amostra-dados/t_cons_reg.csv', sep=';')

In [4]:
#cria um arquivo para cada paciente contendo todos os registros de anotações das consultas realizadas
for npac in cons_reg['npac'].unique():
    text = '\n-----------------------\n\n'.join([t for t in cons_reg[cons_reg['npac'] == npac]['coment_cons_reg']])
    with open(f'{path}cons_regs/{npac}.txt', 'w') as txt_file:
        txt_file.write(text)

In [5]:
cons_reg

,npac,id_cons_reg,coment_cons_reg
0,4162,9016,emrr \n\nEm uso de copaxone ja teve 3 surto...
1,4163,9017,Neurite optica à E em 26 12 13 sem impregnacao...
2,4163,9018,neurite optica a esquerda em 26/12/13 sem imp...
3,4163,9019,neurite optica a e em 26 12 13 sem impregnaca...
4,4164,9020,primeira consulta 44 anos natural de poa ...
...,...,...,...
1238,4824,13119,"QUADRO INALTERADO. \n REFERE QUE, A PARTIR DE ..."
1239,4824,13120,QUADRO PROGRESSIVAMENTE PIOR. \n FEZ INVESTIGA...
1240,4824,13121,REFERE PIORA DO QUADRO COM O USO DO FINGOLIMOD...
1241,4824,16740,Apresentou aumento da gama-GT (não sabe dizer ...


#### Reconhecimento de entidades nomeadas  - NER

#### d4data/biomedical-ner-all
[d4data/biomedical-ner-all](https://huggingface.co/d4data/biomedical-ner-all)

#### facebook/mbart-large-50-many-to-many-mmt
[facebook/mbart-large-50-many-to-many-mmt](https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt)

In [6]:
pipe = pipeline("token-classification", model="d4data/biomedical-ner-all")

In [10]:
text = cons_reg['coment_cons_reg'][1]

print('texto: ', text)
pd.DataFrame(pipe(text)).head(20)

texto:  Neurite optica à E em 26 12 13 sem impregnacao por gadolinio  

rnm normal    anti mog e anti agp4 negativos   

sem novos episodios   rnm 2015 01 sem alteracao    sem novos sintomas            

vertigem periferica retorno em 6 meses para acompanhamento  

plano: se nova rnm se normal alta  

neurite optica em acompanhamento


,entity,score,index,word,start,end
0,B-Coreference,0.987406,1,ne,0,2
1,B-Coreference,0.919065,2,##uri,2,5
2,I-Coreference,0.725892,3,##te,5,7
3,I-Diagnostic_procedure,0.805519,4,optic,8,13
4,I-Coreference,0.741913,5,##a,13,14
5,B-Lab_value,0.405088,7,e,17,18
6,B-Lab_value,0.278259,8,em,19,21
7,B-Lab_value,0.995444,9,26,22,24
8,B-Lab_value,0.717887,10,12,25,27
9,I-Lab_value,0.736103,11,13,28,30


###### Como o modelo utilizado foi treinado para textos médicos em inglês, o NER não apresentou resultado satisfatório para o texto escrito em português. Foi feita uma tentativa de traduzir o texto para inglês e assim avaliar se seria possível utilizar o reconhecimento de entidades nomeadas

In [29]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

model.safetensors: 100%|██████████| 2.44G/2.44G [00:36<00:00, 67.4MB/s]
tokenizer_config.json: 100%|██████████| 529/529 [00:00<?, ?B/s] 
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 90.5MB/s]
special_tokens_map.json: 100%|██████████| 649/649 [00:00<?, ?B/s] 


In [44]:
def translate_to_EN(text):
    tokenizer.src_lang = "pt_XX"
    encoded_text = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )

    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_text

In [45]:
print(text)

translated_text = translate_to_EN(text)
print(translated_text)
pd.DataFrame(pipe(translated_text))

Neurite optica à E em 26 12 13 sem impregnacao por gadolinio  

rnm normal    anti mog e anti agp4 negativos   

sem novos episodios   rnm 2015 01 sem alteracao    sem novos sintomas            

vertigem periferica retorno em 6 meses para acompanhamento  

plano: se nova rnm se normal alta  

neurite optica em acompanhamento
Optical neuritis to E on 26 12 13 no gonadorrhea normal rnm negative anti-mog and anti-agp4 no new episodes rnm 2015 01 no alteration no new symptoms peripheral vertigo return in 6 months to add plan: if new rnm if normal high optical neuritis added


,entity,score,index,word,start,end
0,B-Disease_disorder,0.661541,1,optical,0,7
1,I-Sign_symptom,0.507745,2,ne,8,10
2,I-Sign_symptom,0.541165,3,##uri,10,13
3,I-Disease_disorder,0.628101,4,##tis,13,16
4,B-Biological_structure,0.616918,6,e,20,21
5,B-Date,0.992286,8,26,25,27
6,I-Date,0.999649,9,12,28,30
7,I-Date,0.999826,10,13,31,33
8,B-Sign_symptom,0.982472,12,go,37,39
9,I-Sign_symptom,0.990060,13,##nad,39,42


### A seguir, foram testados dois modelos para resumir ou unir os textos de consultas de um paciente pois verificou-se que um mesmo paciente pode ter diferentes textos de acompanhamento cadastrados. Os textos de acompanhamento são sequenciais, escritos e atualizados a cada consulta do paciente. No entanto isso não significa que cada texto tenha informações adicionais em relação ao texto anterior, os textos podem ser diferentes e não apresentar a característica de serem complementares em relação aos textos anteriores.

#### Falconsai/medical_summarization
[Falconsai/medical_summarization](https://huggingface.co/Falconsai/medical_summarization)

In [46]:
pipe = pipeline("summarization", model="Falconsai/medical_summarization")

In [58]:
text = '\n--------\n\n'.join([t for t in cons_reg[cons_reg['npac'] == 4163]['coment_cons_reg']])

In [59]:
translated_text = ''
for t in cons_reg[cons_reg['npac'] == 4163]['coment_cons_reg']:
    translated_text += '\n--------\n\n' + translate_to_EN(t)

In [63]:
print(text)

#translated_text = translate_to_EN(text)
print('tradução:\n', translated_text)

Neurite optica à E em 26 12 13 sem impregnacao por gadolinio  

rnm normal    anti mog e anti agp4 negativos   

sem novos episodios   rnm 2015 01 sem alteracao    sem novos sintomas            

vertigem periferica retorno em 6 meses para acompanhamento  

plano: se nova rnm se normal alta  

neurite optica em acompanhamento
--------

 neurite optica a esquerda em 26/12/13 sem impregnacao por gadolinio  

rnm crânio normal    

anti mog e anti agp4 negativos   

sem novos episodios    paciente relata que tem episodios de zumbido vertigem e outros     

EN 
beg loc muc  leve nistagmo horizontal  sem dismetria  romberg negativa    

Imp.:
vertigem periferica   sem recorrencia da neurite    

Cond 
prescrevo betadine    
solicito nova rnm    
--------

 neurite optica a e em 26 12 13 sem impregnacao por gadolinio  rnm normal    sem novos episodios    recuperacao quase completa da visao de olho esq  

Cond
coleto sangue para testagem  
oriento encaminhamento via posto de saude 
tradução:


In [64]:
pipe(text, max_length=500, min_length=150)

Your max_length is set to 500, but your input_length is only 399. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=199)


[{'summary_text': 'Neurite optica à E em 26 12 13 sem impregnacao por gadolinio   \xa0\xa0\xa0 \xa0\xa0\xa0\xa0Rnm 2015 01 sem alteracao sem novos sintomas. Neuriteoptica   retorno em 6 meses para acompanhamento  \xa0\xa0\xa0\xa0Rm 2015 01 \xa0Sem novo episodios de zumbido vertigem e outros  \xa0\xa0\xa0\xa0Nomine optica a e em 2612 13 sem \xa0impregnac Sao Paulo \xa0Para testagem  \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Hercules\xa0Carmen\xa0Sotomayor \xa0“Nominate optica”   “Nominate optica \xa0E”.'}]

In [67]:
pipe(translated_text, max_length=500, min_length=100)

Your max_length is set to 500, but your input_length is only 204. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)


[{'summary_text': "Optical neuritis to E on 26 12 13 no gonadorrhea normal rnm negative anti-mog and anti-agp4 no new episodes rnm 2015 01 no alteration no new symptoms peripheral vertigo return in 6 months to add plan: if new rnm if normal high optical neuritis added. Cond prescription betadine I'm asking for new r nm. Cond collected blood for orientation testing orientation via health site. EN beg loc muc mild horizontal nistagland no distortion negative romberg. Imp.: peripheral Vertigo no recurrence of kidney."}]

#### facebook/bart-large-cnn
[facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)

In [68]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

In [69]:
pipe(text, max_length=300, min_length=150)

[{'summary_text': 'Neurite optica à E em 26 12 13 sem impregnacao por gadolinio   \xa0\xa0\xa0 \xa0\xa0\xa0\xa0Rnm 2015 01 sem alteracao sem novos sintomas. Neuriteoptica   retorno em 6 meses para acompanhamento  \xa0\xa0\xa0\xa0Rm 2015 01 \xa0Sem novo episodios de zumbido vertigem e outros  \xa0\xa0\xa0\xa0Nomine optica a e em 2612 13 sem \xa0impregnac Sao Paulo \xa0Para testagem  \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Hercules\xa0Carmen\xa0Sotomayor \xa0“Nominate optica”   “Nominate optica \xa0E”.'}]

In [70]:
pipe(translated_text, max_length=300, min_length=100)

Your max_length is set to 300, but your input_length is only 204. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)


[{'summary_text': "Optical neuritis to E on 26 12 13 no gonadorrhea normal rnm negative anti-mog and anti-agp4 no new episodes rnm 2015 01 no alteration no new symptoms peripheral vertigo return in 6 months to add plan: if new rnm if normal high optical neuritis added. Cond prescription betadine I'm asking for new r nm. Cond collected blood for orientation testing orientation via health site. EN beg loc muc mild horizontal nistagland no distortion negative romberg. Imp.: peripheral Vertigo no recurrence of kidney."}]